# Longformer
### Importing Libraries



In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from transformers import *
import pickle

print('TF version',tf.__version__)

/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_flax_utils.py:24: FutureWarning: Importing `FlaxGenerationMixin` from `src/transformers/generation_flax_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import FlaxGenerationMixin` instead.
  warnings.warn(
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Xformers is not insta

TF version 2.12.0


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
VER=14
Model_path = 'model'
Model_name = 'allenai/longformer-base-4096'

# Load Libraries

In [ ]:
os.mkdir('model')

tokenizer = AutoTokenizer.from_pretrained(Model_name)
tokenizer.save_pretrained('model')
config = AutoConfig.from_pretrained(Model_name)
config.save_pretrained('model')
backbone = TFAutoModel.from_pretrained(Model_name, config=config)
backbone.save_pretrained('model')

Could not locate the tokenizer configuration file, will try to use the model config instead.


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--allenai--longformer-base-4096/snapshots/301e6a42cb0d9976a6d6a26a079fef81c18aa895/config.json
Model config LongformerConfig {
  "_name_or_path": "allenai/longformer-base-4096",
  "attention_mode": "longformer",
  "attention_probs_dropout_prob": 0.1,
  "attention_window": [
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "ignore_attention_mask": false,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 4098,
  "model_type": "longformer",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "onnx_export": false,
  "pad_token_id": 1,
  "sep_token_id": 2,
  "transformers_version": "4.31.0",
  "type_vocab_size": 1,
  "voc

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--allenai--longformer-base-4096/snapshots/301e6a42cb0d9976a6d6a26a079fef81c18aa895/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--allenai--longformer-base-4096/snapshots/301e6a42cb0d9976a6d6a26a079fef81c18aa895/merges.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--allenai--longformer-base-4096/snapshots/301e6a42cb0d9976a6d6a26a079fef81c18aa895/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--allenai--longformer-base-4096/snapshots/301e6a42cb0d9976a6d6a26a079fef81c18aa895/config.json
Model config LongformerConfig {
  "_name_or_path": "allenai/longformer-base-4096",
  "attention_mode": "longformer",
  "attention_probs_dropout_prob

loading weights file tf_model.h5 from cache at /root/.cache/huggingface/hub/models--allenai--longformer-base-4096/snapshots/301e6a42cb0d9976a6d6a26a079fef81c18aa895/tf_model.h5
Input ids are automatically padded from 2 to 512 to be a multiple of `config.attention_window`: 512
Some layers from the model checkpoint at allenai/longformer-base-4096 were not used when initializing TFLongformerModel: ['lm_head']
- This IS expected if you are initializing TFLongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFLongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFLongformerModel were initialized from the model checkpoint at allenai/longformer-base-4096.
If your t

In [ ]:
if os.environ["CUDA_VISIBLE_DEVICES"].count(',') == 0:
    strategy = tf.distribute.get_strategy()
    print('single strategy')
else:
    strategy = tf.distribute.MirroredStrategy()
    print('multiple strategy')
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})

multiple strategy


# Training the model

In [ ]:
train_data = pd.read_csv('/content/gdrive/MyDrive/feedback-prize-2021/train.csv')
print( train_data.shape )
train_data.head()

(144293, 8)


,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...


In [ ]:
print('The train labels are:')
train_data.discourse_type.unique()

The train labels are:


array(['Lead', 'Position', 'Evidence', 'Claim', 'Concluding Statement',
       'Counterclaim', 'Rebuttal'], dtype=object)

In [ ]:
IDS = train_data.id.unique()
print('There are',len(IDS),'train texts.')

There are 15594 train texts.


In [ ]:
from sklearn.utils import resample
# Get the majority class count
majority_count = train_data['discourse_type'].value_counts().max()

# Get all the unique classes
classes = train_data['discourse_type'].unique()

# List to store the upsampled dataframes
df_list = []

# Iterate over all classes and upsample minority classes
for class_type in classes:
    df_class = train_data[train_data.discourse_type == class_type]
    df_class_upsampled = resample(df_class,
                                  replace=True,
                                  n_samples=majority_count,
                                  random_state=42)
    df_list.append(df_class_upsampled)

# Combine majority class with upsampled minority classes
df_upsampled = pd.concat(df_list)

In [ ]:
df_upsampled['discourse_type'].value_counts()

Lead                    50208
Position                50208
Evidence                50208
Claim                   50208
Concluding Statement    50208
Counterclaim            50208
Rebuttal                50208
Name: discourse_type, dtype: int64

In [ ]:
train_data['discourse_type'].value_counts()

Claim                   50208
Evidence                45702
Position                15419
Concluding Statement    13505
Lead                     9305
Counterclaim             5817
Rebuttal                 4337
Name: discourse_type, dtype: int64

In [ ]:
### Tokeninzing the training data
max_len = 1024

tokenizer = AutoTokenizer.from_pretrained(Model_path)
train_tokens = np.zeros((len(IDS),max_len), dtype='int32')
train_attention = np.zeros((len(IDS),max_len), dtype='int32')

lead_b = np.zeros((len(IDS),max_len))
lead_i = np.zeros((len(IDS),max_len))

position_b = np.zeros((len(IDS),max_len))
position_i = np.zeros((len(IDS),max_len))

evidence_b = np.zeros((len(IDS),max_len))
evidence_i = np.zeros((len(IDS),max_len))

claim_b = np.zeros((len(IDS),max_len))
claim_i = np.zeros((len(IDS),max_len))

conclusion_b = np.zeros((len(IDS),max_len))
conclusion_i = np.zeros((len(IDS),max_len))

counterclaim_b = np.zeros((len(IDS),max_len))
counterclaim_i = np.zeros((len(IDS),max_len))

rebuttal_b = np.zeros((len(IDS),max_len))
rebuttal_i = np.zeros((len(IDS),max_len))

train_lens = []
targets_b = [lead_b, position_b, evidence_b, claim_b, conclusion_b, counterclaim_b, rebuttal_b]
targets_i = [lead_i, position_i, evidence_i, claim_i, conclusion_i, counterclaim_i, rebuttal_i]
target_map = {'Lead':0, 'Position':1, 'Evidence':2, 'Claim':3, 'Concluding Statement':4,
             'Counterclaim':5, 'Rebuttal':6}

loading file vocab.json
loading file merges.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json


In [ ]:
assert( np.sum(df_upsampled.groupby('id')['discourse_start'].diff()<=0)==0 )

for id_num in range(len(IDS)):
    if id_num%100==0: print(id_num,', ',end='')

    n = IDS[id_num]
    name = f'/content/gdrive/MyDrive/feedback-prize-2021/train/{n}.txt'
    txt = open(name, 'r').read()
    train_lens.append( len(txt.split()))
    tokens = tokenizer.encode_plus(txt, max_length=max_len, padding='max_length',
                                   truncation=True, return_offsets_mapping=True)
    train_tokens[id_num,] = tokens['input_ids']
    train_attention[id_num,] = tokens['attention_mask']

    offsets = tokens['offset_mapping']
    offset_index = 0
    df = df_upsampled.loc[df_upsampled.id==n]
    for index,row in df.iterrows():
        a = row.discourse_start
        b = row.discourse_end
        if offset_index>len(offsets)-1:
            break
        c = offsets[offset_index][0]
        d = offsets[offset_index][1]
        beginning = True
        while b>c:
            if (c>=a)&(b>=d):
                k = target_map[row.discourse_type]
                if beginning:
                    targets_b[k][id_num][offset_index] = 1
                    beginning = False
                else:
                    targets_i[k][id_num][offset_index] = 1
            offset_index += 1
            if offset_index>len(offsets)-1:
                break
            c = offsets[offset_index][0]
            d = offsets[offset_index][1]

0 , 100 , 200 , 300 , 400 , 500 , 600 , 700 , 800 , 900 , 1000 , 1100 , 1200 , 1300 , 1400 , 1500 , 1600 , 1700 , 1800 , 1900 , 2000 , 2100 , 2200 , 2300 , 2400 , 2500 , 2600 , 2700 , 2800 , 2900 , 3000 , 3100 , 3200 , 3300 , 3400 , 3500 , 3600 , 3700 , 3800 , 3900 , 4000 , 4100 , 4200 , 4300 , 4400 , 4500 , 4600 , 4700 , 4800 , 4900 , 5000 , 5100 , 5200 , 5300 , 5400 , 5500 , 5600 , 5700 , 5800 , 5900 , 6000 , 6100 , 6200 , 6300 , 6400 , 6500 , 6600 , 6700 , 6800 , 6900 , 7000 , 7100 , 7200 , 7300 , 7400 , 7500 , 7600 , 7700 , 7800 , 7900 , 8000 , 8100 , 8200 , 8300 , 8400 , 8500 , 8600 , 8700 , 8800 , 8900 , 9000 , 9100 , 9200 , 9300 , 9400 , 9500 , 9600 , 9700 , 9800 , 9900 , 10000 , 10100 , 10200 , 10300 , 10400 , 10500 , 10600 , 10700 , 10800 , 10900 , 11000 , 11100 , 11200 , 11300 , 11400 , 11500 , 11600 , 11700 , 11800 , 11900 , 12000 , 12100 , 12200 , 12300 , 12400 , 12500 , 12600 , 12700 , 12800 , 12900 , 13000 , 13100 , 13200 , 13300 , 13400 , 13500 , 13600 , 13700 , 13800 , 

In [ ]:
saved_path = '/content/gdrive/MyDrive/feedback-prize-2021/tokens/'

targets = np.zeros((len(IDS),max_len,15), dtype='int32')
for k in range(7):
    targets[:,:,2*k] = targets_b[k]
    targets[:,:,2*k+1] = targets_i[k]
targets[:,:,14] = 1-np.max(targets,axis=-1)

np.save(saved_path + f'targets_{max_len}', targets)
np.save(saved_path + f'tokens_{max_len}', train_tokens)
np.save(saved_path + f'attention_{max_len}', train_attention)
print('NER tokens are saved')

Saved NER tokens


In [ ]:
targets = np.load(saved_path + f'targets_{max_len}.npy')
train_tokens = np.load(saved_path + f'tokens_{max_len}.npy')
train_attention = np.load(saved_path + f'attention_{max_len}.npy')
print('Loaded NER tokens')

Loaded NER tokens


# Model

In [ ]:
def build_model():

    tokens = tf.keras.layers.Input(shape=(max_len,), name = 'tokens', dtype=tf.int32)
    attention = tf.keras.layers.Input(shape=(max_len,), name = 'attention', dtype=tf.int32)

    config = AutoConfig.from_pretrained(Model_path+'/config.json')
    backbone = TFAutoModel.from_pretrained(Model_path+'/tf_model.h5', config=config)

    x = backbone(tokens, attention_mask=attention)
    x = tf.keras.layers.Dense(256, activation='relu')(x[0])
    x = tf.keras.layers.Dense(15, activation='softmax', dtype='float32')(x)

    model = tf.keras.Model(inputs=[tokens,attention], outputs=x)
    model.compile(optimizer = tf.keras.optimizers.Adam(lr = 1e-4),
                  loss = [tf.keras.losses.CategoricalCrossentropy()],
                  metrics = [tf.keras.metrics.CategoricalAccuracy()])

    return model

with strategy.scope():
    model = build_model()

In [ ]:
## Hyper Parameters
EPOCHS = 5
BATCH_SIZE = 4
LRS = [0.25e-4, 0.25e-4, 0.25e-4, 0.25e-4, 0.25e-5]
def lrfn(epoch):
    return LRS[epoch]
lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)

In [ ]:
# Train-Valid split 80-20
np.random.seed(42)
train_idx = np.random.choice(np.arange(len(IDS)),int(0.8*len(IDS)),replace=False)
valid_idx = np.setdiff1d(np.arange(len(IDS)),train_idx)
np.random.seed(None)
print('Train size',len(train_idx),', Valid size',len(valid_idx))

Train size 12475 , Valid size 3119


In [ ]:
load_model_path = True
if load_model_path:
    model.load_weights(saved_path + f'/long_v{VER}.h5')


else:
    history = model.fit(x = [train_tokens[train_idx,], train_attention[train_idx,]],
          y = targets[train_idx,],
          validation_data = ([train_tokens[valid_idx,], train_attention[valid_idx,]],
                             targets[valid_idx,]),
          callbacks = [lr_callback],
          epochs = EPOCHS,
          batch_size = BATCH_SIZE,
          verbose = 2)

    model.save_weights(saved_path + f'long_v{VER}.h5')

    with open('/content/gdrive/MyDrive/feedback-prize-2021/tokens/history.pkl', 'wb') as f:
        pickle.dump(history.history, f)

In [ ]:
import matplotlib.pyplot as plt
train_loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(train_loss) + 1)

plt.plot(epochs, train_loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Longformer Training vs. Validation Loss')
plt.legend()
plt.show()

# Model Prediction

In [ ]:
pred = model.predict([train_tokens[valid_idx,], train_attention[valid_idx,]],
                  batch_size=16, verbose=2)
print('predictions shape:',pred.shape)
predicts = np.argmax(pred,axis=-1)

target_map = {0:'Lead', 1:'Position', 2:'Evidence', 3:'Claim', 4:'Concluding Statement',
             5:'Counterclaim', 6:'Rebuttal', 7:'blank'}

In [ ]:
#prediction function
def pred_fun(dataset='df_upsampled', verbose=True, text_ids=IDS[valid_idx], preds=predicts):
    all_predictions = []

    for id_num in range(len(preds)):

        if (id_num%100==0)&(verbose):
            print(id_num,', ',end='')
        n = text_ids[id_num]

        # token positions to char
        name = f'/content/gdrive/MyDrive/feedback-prize-2021/train/{n}.txt'
        txt = open(name, 'r').read()
        tokens = tokenizer.encode_plus(txt, max_length=max_len, padding='max_length',
                                   truncation=True, return_offsets_mapping=True)
        off = tokens['offset_mapping']
        w = []
        blank = True
        for i in range(len(txt)):
            if (txt[i]!=' ')&(txt[i]!='\n')&(txt[i]!='\xa0')&(txt[i]!='\x85')&(blank==True):
                w.append(i)
                blank=False
            elif (txt[i]==' ')|(txt[i]=='\n')|(txt[i]=='\xa0')|(txt[i]=='\x85'):
                blank=True
        w.append(1e6)

        # mapping from tokens to words
        word_map = -1 * np.ones(max_len,dtype='int32')
        w_i = 0
        for i in range(len(off)):
            if off[i][1]==0: continue
            while off[i][0]>=w[w_i+1]: w_i += 1
            word_map[i] = int(w_i)

        pred = preds[id_num,]/2.0

        i = 0
        while i<max_len:
            prediction = []
            start = pred[i]
            if start in [0,1,2,3,4,5,6,7]:
                prediction.append(word_map[i])
                i += 1
                if i>=max_len: break
                while pred[i]==start+0.5:
                    if not word_map[i] in prediction:
                        prediction.append(word_map[i])
                    i += 1
                    if i>=max_len: break
            else:
                i += 1
            prediction = [x for x in prediction if x!=-1]
            if len(prediction)>4:
                all_predictions.append( (n, target_map[int(start)],
                                ' '.join([str(x) for x in prediction]) ) )
    df = pd.DataFrame(all_predictions)
    df.columns = ['id','class','predictionstring']

    return df

In [ ]:
final_preds = pred_fun( dataset='train', verbose=True, text_ids=IDS[valid_idx] )
final_preds.head()

0 , 100 , 200 , 300 , 400 , 500 , 600 , 700 , 800 , 900 , 1000 , 1100 , 1200 , 1300 , 1400 , 1500 , 1600 , 1700 , 1800 , 1900 , 2000 , 2100 , 2200 , 2300 , 2400 , 2500 , 2600 , 2700 , 2800 , 2900 , 3000 , 3100 , 

,id,class,predictionstring
0,A8445CABFECE,Position,3 4 5 6 7 8 9 10 11 12 13 14
1,A8445CABFECE,Claim,15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 3...
2,A8445CABFECE,Evidence,36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 5...
3,A8445CABFECE,Evidence,112 113 114 115 116 117 118 119 120 121 122 12...
4,A8445CABFECE,Concluding Statement,177 178 179 180 181 182 183 184 185 186 187 18...


In [ ]:
print('Classes present in final predictions:')
final_preds['class'].unique()

Classes present in final predictions:


array(['Position', 'Claim', 'Evidence', 'Concluding Statement', 'Lead',
       'Counterclaim', 'Rebuttal'], dtype=object)

# Evaluating Model

In [ ]:
## Calculating F1-Score, Precision, Recall
def calc_overlap(row):

    set_pred = set(row.predictionstring_pred.split(' '))
    set_gt = set(row.predictionstring_gt.split(' '))
    # Length of each and intersection
    len_gt = len(set_gt)
    len_pred = len(set_pred)
    inter = len(set_gt.intersection(set_pred))
    overlap_1 = inter / len_gt
    overlap_2 = inter/ len_pred
    return [overlap_1, overlap_2]


def eval_score(pred_df, gt_df):
    gt_df = gt_df[['id','discourse_type','predictionstring']] \
        .reset_index(drop=True).copy()
    pred_df = pred_df[['id','class','predictionstring']] \
        .reset_index(drop=True).copy()
    pred_df['pred_id'] = pred_df.index
    gt_df['gt_id'] = gt_df.index
    # Step 1. all ground truths and predictions for a given class are compared.
    joined = pred_df.merge(gt_df,
                           left_on=['id','class'],
                           right_on=['id','discourse_type'],
                           how='outer',
                           suffixes=('_pred','_gt')
                          )
    joined['predictionstring_gt'] = joined['predictionstring_gt'].fillna(' ')
    joined['predictionstring_pred'] = joined['predictionstring_pred'].fillna(' ')

    joined['overlaps'] = joined.apply(calc_overlap, axis=1)

    joined['overlap1'] = joined['overlaps'].apply(lambda x: eval(str(x))[0])
    joined['overlap2'] = joined['overlaps'].apply(lambda x: eval(str(x))[1])


    joined['potential_TP'] = (joined['overlap1'] >= 0.5) & (joined['overlap2'] >= 0.5)
    joined['max_overlap'] = joined[['overlap1','overlap2']].max(axis=1)
    tp_pred_ids = joined.query('potential_TP') \
        .sort_values('max_overlap', ascending=False) \
        .groupby(['id','predictionstring_gt']).first()['pred_id'].values

    fp_pred_ids = [p for p in joined['pred_id'].unique() if p not in tp_pred_ids]

    matched_gt_ids = joined.query('potential_TP')['gt_id'].unique()
    unmatched_gt_ids = [c for c in joined['gt_id'].unique() if c not in matched_gt_ids]

    TP = len(tp_pred_ids)
    FP = len(fp_pred_ids)
    FN = len(unmatched_gt_ids)
    precision = TP / (TP + FP) if TP + FP > 0 else 0
    recall = TP / (TP + FN) if TP + FN > 0 else 0
    my_f1_score = TP / (TP + 0.5*(FP+FN))
    return my_f1_score,precision,recall

In [ ]:
df_valid = df_upsampled.loc[df_upsampled['id'].isin(IDS[valid_idx])]

In [ ]:
f1s = []
precisions = []
recalls = []
classs = final_preds['class'].unique()

for c in classs:
    pred_df = final_preds.loc[final_preds['class']==c].copy()
    gt_df = df_valid.loc[df_valid['discourse_type']==c].copy()
    f1, precision, recall = eval_score(pred_df, gt_df)
    print(f"Class {c} - F1: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")
    f1s.append(f1)
    precisions.append(precision)
    recalls.append(recall)

print()
print(f'Overall - F1: {np.mean(f1s):.4f}, Precision: {np.mean(precisions):.4f}, Recall: {np.mean(recalls):.4f}')

Class Position - F1: 0.7327, Precision: 0.7849, Recall: 0.6870
Class Claim - F1: 0.6549, Precision: 0.7270, Recall: 0.5958
Class Evidence - F1: 0.7294, Precision: 0.7820, Recall: 0.6834
Class Concluding Statement - F1: 0.8258, Precision: 0.8446, Recall: 0.8079
Class Lead - F1: 0.8655, Precision: 0.8617, Recall: 0.8693
Class Counterclaim - F1: 0.6369, Precision: 0.6676, Recall: 0.6090
Class Rebuttal - F1: 0.5538, Precision: 0.5902, Recall: 0.5217

Overall - F1: 0.7142, Precision: 0.7511, Recall: 0.6820
